In [15]:
import os
import sys
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from dotenv import load_dotenv
_ = load_dotenv()

from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import AzureChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun

In [16]:
from tools.GetPayslipTool import get_payslip
from tools.GetReservationsTool import get_reservations
from tools.GetContractsTool import get_contracts
from tools.GetScheduleTool import get_schedule

In [17]:
# Azure AI Foundry configuration
model = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-12-01-preview",
    azure_deployment="gpt-4.1",  # Your deployment name
    model="gpt-4.1",
    temperature=0
)

In [18]:
model.invoke([SystemMessage(content="Ready for working?")]).content

"Yes, I'm ready to help! What can I assist you with today?"

## Import Prompts

In [19]:


# Load the intent classifier prompt from the text file
with open(os.path.join(parent_dir, "prompts", "intent_classifier_prompt.txt"), "r") as f:
    intent_classifier_prompt = f.read()

# Load the action execution prompt from the text file
with open(os.path.join(parent_dir, "prompts", "action_execution_prompt.txt"), "r") as f:
    action_execution_prompt = f.read()

In [20]:
intent_classifier_prompt

'You are an intent classifier for the ChatSupervisor agent, the central brain of a worker assistant. Your primary responsibility is to analyze the user\'s latest message in the context of the chat history and determine the user\'s primary intent.\n\nYour response must be only the name of the single, most appropriate agent or tool to handle this intent. Do not provide any other text, explanation, or preamble. Respond with the agent name on a single line, with no extra whitespace, punctuation, or formatting.\n\nThese are your only valid routing options:\n\n1.  InformationRetrievalAgent\n    - Purpose: To answer generic questions about work policies, procedures, or "how-to" guides using the internal knowledge base.\n    - Examples:\n        - "How do I call in sick?"\n        - "What are the company rules for overtime?"\n        - "How are reservations calculated?"\n\n2.  ActionExecutionAgent\n    - Purpose: To handle personalized requests that require fetching the user\'s own specific da

In [21]:
action_execution_prompt

"You are the Action Execution Agent, an expert in personalized data retrieval. \nYour sole purpose is to analyze the user's request and select the single, most appropriate tool to fetch their personal data (schedule, payslip, contract, etc.). \nYou must accurately extract all necessary parameters (like dates or specific document types) from the user's message.\nAfter execution, return the tool output; **do not generate a conversational response**."

## Agent Set-up

In [22]:
# Agent State
class AgentState(TypedDict):
   messages: Annotated[list[AnyMessage], "Conversation messages"]
   next_action: Annotated[str, "Next action to take"]
   retrieved_data: Annotated[str, "Data retrieved from tools"]
   error_message: Annotated[str, "Error message if any"]
   attempts: Annotated[int, "Number of attempts made"]
   max_retries: Annotated[int, "Maximum number of retries allowed"]

In [31]:
from langchain_core.runnables import Runnable
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.prebuilt import ToolNode

class ActionExecutionAgent:
    def __init__(self, llm_model, system_prompt):
        # 1. Define the tools available to this agent
        self.tools = [
            get_schedule,
            get_payslip,
            get_contracts,
            get_reservations,
        ]
        
        # 2. Bind the LLM to the tools for function calling
        #    This allows the model to return a structured 'tool_call' object
        self.llm_with_tools: Runnable = llm_model.bind_tools(self.tools)
        # 3. Define the tool executor
        # 3. Define the tool executor
        #    This component handles executing the tool calls requested by the LLM
        self.tool_executor = ToolNode(self.tools)
        # 4. Agent's system prompt (for reasoning/tool selection)
        self.system_prompt = system_prompt 

    def run(self, state: AgentState) -> dict:
        """
        The method executed as the LangGraph node. It acts as the agent loop.
        """
        # 1. Get the latest message (the specific personalized request)
        last_message = state["messages"][-1]
        print("last_message: ", last_message)
        
        # 2. Add the agent's specific system prompt to the call
        #    This prompt guides the agent to select and execute the single best tool
        messages = [SystemMessage(content=self.system_prompt)] + [last_message]
        print("messages: ", messages)
        
        # 3. Call the LLM to decide on the tool and parameters
        agent_response = self.llm_with_tools.invoke(messages)
        print("agent_response:", agent_response)
        # 4. Check for tool call
        if agent_response.tool_calls:
            # 5. Execute the tool directly using ToolNode
            #    ToolNode can handle the agent_response with tool_calls directly
            tool_output = self.tool_executor.invoke({"messages": [agent_response]})
            print("tool_output:", tool_output)
            
            # 6. Extract the actual tool result from the tool message
            if tool_output and "messages" in tool_output:
                tool_result = tool_output["messages"][-1].content
                return {"retrieved_data": tool_result}
            
            print("tool_result:", tool_result)
            # 7. Update state with data
            return {"retrieved_data": str(tool_output)}
            # 7. Update state with data
            return {"retrieved_data": str(tool_output)}
            # Failsafe: If the agent hallucinates or refuses to use a tool, route to Fallback
            return {"error": "Agent failed to execute tool call."}

In [32]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END

# Ensure AgentState is defined (as discussed previously)
# class AgentState(TypedDict):
#     messages: Annotated[List[BaseMessage], operator.add]
#     next_action: str
#     retrieved_data: str
#     error: str

class ChatSupervisorAgent:
    def __init__(self, model, checkpointer=None):
        self.model = model
        self.system = intent_classifier_prompt
        
        self.action_agent_instance = ActionExecutionAgent(
            llm_model=model,
            system_prompt=action_execution_prompt
        ) 

        graph = StateGraph(AgentState)

        # --- 1. Define Nodes ---
        graph.add_node("classify_intent", self.classify_intent_node)
        graph.add_node("action_execution_agent", self.action_agent_instance.run)
        graph.add_node("information_retrieval_agent", self.information_retrieval_agent)
        graph.add_node("fallback_tool", self.fallback_tool)
        graph.add_node("answer_agent", self.answer_agent) # REQUIRED by architecture

        # --- 2. Define The Entry Point ---
        graph.set_entry_point("classify_intent")

        # --- 3. Define The Conditional Edge (The Router) ---
        graph.add_conditional_edges(
            "classify_intent", 
            self.route_intent, 
            {
                "ActionExecutionAgent": "action_execution_agent",
                "InformationRetrievalAgent": "information_retrieval_agent",
                "FallbackTool": "fallback_tool",
            }
        )

        # --- 4. Define Normal Edges (Enforcing ToV) ---
        graph.add_edge("action_execution_agent", "answer_agent")
        graph.add_edge("information_retrieval_agent", "answer_agent")
        graph.add_edge("fallback_tool", "answer_agent")
        
        # AnswerAgent finishes the run
        graph.add_edge("answer_agent", END)

        # 5. Compile
        self.graph = graph.compile(checkpointer=checkpointer)

    def classify_intent_node(self, state: AgentState):
        '''Reads the user query and classifies the primary action.'''
        messages = [SystemMessage(content=self.system)] + state['messages']
        response = self.model.invoke(messages)
        intent = response.content.strip()

        # Failsafe for valid routing
        valid_routes = ["ActionExecutionAgent", "InformationRetrievalAgent", "FallbackTool"]
        if intent not in valid_routes:
            print(f"Warning: Invalid intent '{intent}'. Defaulting to Fallback.")
            intent = "FallbackTool"

        print(f"Intent classified as: {intent}")
        
        # Return the update to the state
        return {"next_action": intent}

    def route_intent(self, state: AgentState):
        '''Reads the decision made by the intent classifier node'''
        return state["next_action"]

    # --- SPECIALISTS ---
    def action_execution_agent(self, state: AgentState):
        '''...'''
        print("---Start Action Execution Agent---")
        # Logic to call your tools...
        retrieved_data = ""
        
        
        # return {"retrieved_data": "Schedule for next week..."}
        return {"retrieved_data": retrieved_data}

    def information_retrieval_agent(self, state: AgentState):
        '''Retrieves information from the Sharepoint KnowledgeBase'''
        print("---Start Information Retrieval Agent---")
        # Logic to call RAG...
        return {"retrieved_data": "Sick leave policy details..."}

    def fallback_tool(self, state: AgentState):
        print("---Start Fallback Tool---")
        return {"error": "Could not resolve query."}

    # --- ANSWER AGENT (Final Response) ---
    def answer_agent(self, state: AgentState):
        print("---Start Answer Agent---")
        # return {"messages": [AIMessage(content="Final response to user")]}
        return {}

In [33]:
agent = ChatSupervisorAgent(model=model)

In [34]:
# message = "what's my schedule for next week?"
message = "How can I call in sick?"
message = "What's my availability next week?"
# message = "What's Trump's first name?"

In [35]:
test_state = {
    "messages": [HumanMessage(content=message)],
    "next_action": "",
    "retrieved_data": "",
    "error_message": "",
    "attempts": 0,
    "max_retries": 3
}

result = agent.graph.invoke(test_state)

Intent classified as: ActionExecutionAgent
last_message:  content="What's my availability next week?" additional_kwargs={} response_metadata={}
messages:  [SystemMessage(content="You are the Action Execution Agent, an expert in personalized data retrieval. \nYour sole purpose is to analyze the user's request and select the single, most appropriate tool to fetch their personal data (schedule, payslip, contract, etc.). \nYou must accurately extract all necessary parameters (like dates or specific document types) from the user's message.\nAfter execution, return the tool output; **do not generate a conversational response**.", additional_kwargs={}, response_metadata={}), HumanMessage(content="What's my availability next week?", additional_kwargs={}, response_metadata={})]
agent_response: content='' additional_kwargs={'tool_calls': [{'id': 'call_2PfU2esAQ36H2JVliEZ4YeoT', 'function': {'arguments': '{"employee_number":"<employee_number>","start_date":"2024-06-10","end_date":"2024-06-16"}', 

In [37]:
result

{'messages': [HumanMessage(content="What's my availability next week?", additional_kwargs={}, response_metadata={})],
 'next_action': 'ActionExecutionAgent',
 'retrieved_data': [],
 'error_message': '',
 'attempts': 0,
 'max_retries': 3}

In [42]:
agent.__getstate__

<function ChatSupervisorAgent.__getstate__()>